In [46]:
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import torch
import transformers as ppb
import warnings
import string
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
warnings.filterwarnings('ignore')

In [2]:
!pip install tensorflow as tf

     |████████████████████████████████| 421.8MB 123kB/s  eta 0:00:01
     |████████████████████████████████| 51kB 23.2MB/s eta 0:00:01
     |████████████████████████████████| 112kB 44.8MB/s eta 0:00:01
     |████████████████████████████████| 26.1MB 53.2MB/s eta 0:00:01
     |████████████████████████████████| 71kB 32.6MB/s eta 0:00:01
     |████████████████████████████████| 3.8MB 15.9MB/s eta 0:00:01
     |████████████████████████████████| 450kB 53.6MB/s eta 0:00:01
     |████████████████████████████████| 51kB 28.6MB/s eta 0:00:01
     |████████████████████████████████| 2.4MB 55.8MB/s eta 0:00:01
     |████████████████████████████████| 61kB 20.6MB/s eta 0:00:01
     |████████████████████████████████| 327kB 38.3MB/s eta 0:00:01
     |████████████████████████████████| 92kB 38.8MB/s eta 0:00:01
     |████████████████████████████████| 81kB 37.6MB/s eta 0:00:01
     |████████████████████████████████| 163kB 56.6MB/s eta 0:00:01
     |████████████████████████████████| 153kB 56.0MB/s eta 0:00:0

In [15]:
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.models import Sequential

In [3]:
train=pd.read_csv('train.csv')

In [4]:
test=pd.read_csv('test.csv')

In [5]:
submission=pd.read_csv('sample_submission.csv')

In [6]:
features=np.load('fetures_distill.npy')

In [7]:
features_test=np.load('features_test.npy')

In [8]:
features.shape

(7613, 768)

In [9]:
labels=train['target']

In [10]:
labels.shape

(7613,)

In [16]:
model=Sequential()
model.add(Input(shape=(768,), dtype=tf.float64, name="input_feature"))
model.add(Dense(384, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer = optimizer , loss = "binary_crossentropy", metrics=["accuracy"])


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [17]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [27]:
epochs = 100
batch_size = 86


In [19]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 384)               295296    
_________________________________________________________________
dropout_1 (Dropout)          (None, 384)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 385       
Total params: 295,681
Trainable params: 295,681
Non-trainable params: 0
_________________________________________________________________


In [22]:
X_train, X_val, Y_train, Y_val=train_test_split(features, labels, test_size=0.1)

In [23]:
print(X_train.shape)
print(X_val.shape)

(6851, 768)
(762, 768)


In [28]:
trained_model = model.fit(
    X_train, Y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_val, Y_val),
    verbose=2
)

Train on 6851 samples, validate on 762 samples
Epoch 1/100
6851/6851 - 0s - loss: 0.1184 - accuracy: 0.9559 - val_loss: 0.7862 - val_accuracy: 0.7913
Epoch 2/100
6851/6851 - 0s - loss: 0.1119 - accuracy: 0.9581 - val_loss: 0.7425 - val_accuracy: 0.7835
Epoch 3/100
6851/6851 - 0s - loss: 0.1140 - accuracy: 0.9583 - val_loss: 0.7382 - val_accuracy: 0.7913
Epoch 4/100
6851/6851 - 0s - loss: 0.1154 - accuracy: 0.9546 - val_loss: 0.7525 - val_accuracy: 0.7887
Epoch 5/100
6851/6851 - 0s - loss: 0.1108 - accuracy: 0.9603 - val_loss: 0.7767 - val_accuracy: 0.8071
Epoch 6/100
6851/6851 - 0s - loss: 0.1065 - accuracy: 0.9594 - val_loss: 0.8262 - val_accuracy: 0.8058
Epoch 7/100
6851/6851 - 0s - loss: 0.1027 - accuracy: 0.9593 - val_loss: 0.8514 - val_accuracy: 0.7612
Epoch 8/100
6851/6851 - 0s - loss: 0.1104 - accuracy: 0.9590 - val_loss: 0.7803 - val_accuracy: 0.7992
Epoch 9/100
6851/6851 - 0s - loss: 0.1019 - accuracy: 0.9618 - val_loss: 0.7976 - val_accuracy: 0.7966
Epoch 10/100
6851/6851 - 0

In [29]:
predictions=model.predict(features_test)

In [30]:
print(predictions.shape)

(3263, 1)


In [31]:
submission['target']= predictions.round().astype(int)
submission.to_csv('submission_NN.csv', index=False)

In [32]:
from IPython.display import FileLink
FileLink('submission_NN.csv')

/storage/submission_NN.csv

In [33]:
clf=LogisticRegression()
clf.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [34]:
clf.score(X_val, Y_val)

0.8228346456692913

In [35]:
prediction=clf.predict(features_test)

In [36]:
submission['target']=prediction.round().astype(int)
submission.to_csv('submission_LR.csv', index=False)

In [37]:
FileLink('submission_LR.csv')

/storage/submission_LR.csv

In [39]:
GNB= GaussianNB()
GNB.fit(X_train,Y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [40]:
GNB.score(X_val, Y_val)

0.7637795275590551

In [41]:
DTC= DecisionTreeClassifier(criterion='gini', 
                             min_samples_split=10,min_samples_leaf=1,
                             max_features='auto')
DTC.fit(X_train,Y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [42]:
DTC.score(X_val, Y_val)

0.6811023622047244

In [43]:
model= AdaBoostClassifier()
model.fit(X_train,Y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=None)

In [44]:
model.score(X_val, Y_val)

0.7742782152230971

In [47]:
model= LinearDiscriminantAnalysis()
model.fit(X_train,Y_train)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)

In [48]:
model.score(X_val, Y_val)

0.8228346456692913

In [49]:
submission['target']=prediction.round().astype(int)
submission.to_csv('submission_LDA.csv', index=False)
FileLink('submission_LDA.csv')

/storage/submission_LDA.csv

In [50]:
model= GradientBoostingClassifier()
model.fit(X_train,Y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [51]:
model.score(X_val, Y_val)

0.8018372703412073

In [52]:
model = RandomForestClassifier(criterion='gini', n_estimators=700,
                             min_samples_split=10,min_samples_leaf=1,
                             max_features='auto',oob_score=True,
                             random_state=1,n_jobs=-1)
model.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=700,
                       n_jobs=-1, oob_score=True, random_state=1, verbose=0,
                       warm_start=False)

In [53]:
model.score(X_val, Y_val)

0.8070866141732284

In [54]:
model = SVC()
model.fit(X_train,Y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [56]:
model.score(X_val, Y_val)

0.8018372703412073

In [57]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
random_forest.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=1,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [59]:
random_forest.score(X_val, Y_val)

0.7979002624671916